In [3]:
import spacy
#import gensim
import pandas as pd
import sklearn
import numpy as np
from scipy.spatial.distance import cosine
import random

In [4]:
en_nlp = spacy.en.English()

In [5]:
doc = en_nlp(u"Apples and oranges are similar. Boots and hippos aren't.")

In [6]:
apples = doc[0]

In [7]:
len(apples.vector)

300

In [8]:
apples.ent_type_

'ORG'

In [49]:
def iter_products(docs):
    for doc in docs:
        for ent in doc.ents:
            #if ent.label_ == 'LOCATION':
            yield ent,ent.label_

In [55]:
def get_pos(doc):
    for token in doc:
        #if ent.label_ == 'LOCATION':
        yield token,token.pos_,token.tag_

In [51]:
list(iter_products([en_nlp(u"my name is Gopal?")]))

[(Gopal, 'PERSON')]

In [61]:
list(get_pos(en_nlp("do you know my name")))

[(do, 'VERB', 'VBP'),
 (you, 'PRON', 'PRP'),
 (know, 'VERB', 'VB'),
 (my, 'ADJ', 'PRP$'),
 (name, 'NOUN', 'NN')]

In [11]:
#word2vec_model = gensim.models.word2vec.Word2Vec()

In [12]:
#wordvecs = word2vec_model.load_word2vec_format('./wiki.bin', binary=True)

In [219]:
def get_vec(sentence):
    vec = np.zeros(300)
    for word in sentence:
        if word in wordvecs:
            vec += wordvecs[word]
    return vec
    
    

In [17]:
def spacy_get_vec(sentence):
    vec = np.zeros(300)
    doc = en_nlp((sentence))
    for word in doc:
        vec += word.vector
    return vec

In [14]:
apple = doc[0]
orange = doc[2]
1 - cosine(apple.vector, orange.vector)

0.78579902648925781

In [44]:
1 - cosine(spacy_get_vec('can you tell me my name'), spacy_get_vec('what is your name'))

0.77818442574738045

In [299]:
1 - cosine(wordvecs['banana'], wordvecs['oranges'])

0.53536665166534192

In [18]:
lines = open('./intents.txt').readlines()

In [19]:
vecs = []
intents = []
for line in lines:
    tokens = line.split(',')
    sentence = tokens[0]
    intent = tokens[1]
    if intent[-1] == '\n':
        intent = intent[:-1]
    vecs.append(spacy_get_vec(sentence))
    intents.append(intent)

In [20]:
df = pd.DataFrame(vecs, columns=['vec_%d' % i for i in range(300)])

In [21]:
df['intents'] = intents

In [22]:
df.intents = df.intents.astype('category')

In [23]:
X = df.iloc[:, :-1].values

In [24]:
y = df.iloc[:,-1:].values.ravel()

In [26]:
from sklearn.linear_model import LogisticRegression
logit_model = LogisticRegression()
logit_model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
logit_model.score(X, y)

0.94444444444444442

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

In [29]:
gradboost = GradientBoostingClassifier(n_estimators=500, max_depth=20)

In [30]:
gradboost.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=20,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [31]:
gradboost.score(X, y)

1.0

In [33]:
prediction = gradboost.predict(spacy_get_vec('is there a hailstorm outside'))

/home/abhinav/miniconda3/envs/bots/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [34]:
prediction[0]


'weather'

In [35]:
logit_model.predict_proba(spacy_get_vec('is there a hailstorm outside').reshape(1,-1))

array([[ 0.10539365,  0.12896656,  0.14595851,  0.61968127]])

In [36]:
from sklearn.externals import joblib
joblib.dump(gradboost, 'gradboost.pkl')

['gradboost.pkl']